<a href="https://colab.research.google.com/github/ammobam/Display_SensorData/blob/main/210803_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%83%90%EC%83%89(5)%EC%BB%AC%EB%9F%BC%ED%86%B5%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **디스플레이 생산라인 센서 데이터 분석

## 주제 : 불량품에 영향을 주는 요인 찾기
- 결과 : "ㅇㅇ센서에서 ㅁㅁ면 불량품"
1. 주요 요인 피처 5개 추출
2. 불량품/양품 구분하는 모델도 찾아야 함


- 머신러닝 이용
---

In [39]:
# 데이터 불러오기
import pandas as pd
fact_data = pd.read_csv('/content/drive/MyDrive/나무플래닛/1. 데이터_디스플레이/factory_glass_2016.zip', encoding='cp949')
#fact_data.info()

# dat...date.name. 열을 인덱스로 설정함
# 유일한 object 컬럼으로, 컬럼에서 제외함
# 이제 데이터는 모두 숫자형으로, 연산 가능함
fact_data = fact_data.set_index('dat...date.name.', drop=True)
fact_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 840 entries, LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101 to R.Vac
dtypes: float64(834), int64(6)
memory usage: 52.3+ MB


In [40]:
# 결측치가 많은 컬럼, 행 삭제
# 전체 행 8145개 중 nan값이 8145개인 컬럼은 삭제함
fact_data = fact_data.dropna(how='all', axis=1)
# 컬럼수: 840개 -> 833개

# 행별 결측치 확인
print(fact_data[fact_data.isnull().any(axis=1)])
drop_index = list(fact_data[fact_data.isnull().any(axis=1)].index)
print(drop_index)

                  LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101  ...  R.Vac
dat...date.name.                                     ...       
2016-06-24 04                              1.467846  ...    0.0
2016-10-13 09                             -0.846817  ...    0.0
2016-12-27 17                             -0.846817  ...    0.0

[3 rows x 833 columns]
['2016-06-24 04', '2016-10-13 09', '2016-12-27 17']


In [41]:
# NaN 있는 행 - ffill로 값 채우기
fact_data.fillna(method='ffill', inplace=True)
# 확인
print(fact_data[df_origin.isnull().any(axis=1)])

Empty DataFrame
Columns: [LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101, LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102, LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103, LEHR.DRV.MTR..M4.1.SPD.1WSI40104, LEHR.DRV.MTR..M4.2.SPD.1WSI40105, LEHR.DRV.MTR..M5.1.SPD.1WSI40106, LEHR.DRV.MTR..M5.2.SPD.1WSI40107, LEHR.DRV.MTR..M6.1.SPD.1WSI40108, LEHR.DRV.MTR..M6.2.SPD.1WSI40109, LEHR.DRV.MTR..M7.1.SPD.1WSI40110, LEHR.DRV.MTR..M7.2.SPD.1WSI40111, LEHR.DRV.MTR..M8.1.SPD.1WSI40112, LEHR.DRV.MTR..M8.2.SPD.1WSI40113, LEHR.DRV.MTR..M9.1.SPD.1WSI40114, LEHR.DRV.MTR..M9.2.SPD.1WSI40115, DRIVE.MOTOR.M10.1.SPEED.1WSI40116, LEHR.DRV.MTR..M10.2.SPD.1WSI40117, LEHR.DRV.MTR..M11.1.SPD.1WSI40118, DRIVE.MOTOR.M11.2.SPEED.1WSI40119, LEHR.DRV.MTR..M12.1.SPD.1WSI40120, LEHR.DRV.MTR..M12.2.SPD.1WSI40121, LEHR.DRV.MTR..M13.1.SPD.1WSI40122, LEHR.DRV.MTR..M13.2.SPD.1WSI40123, LEHR.DRV.MTR..M14.1.SPD.1WSI40124, LEHR.DRV.MTR..M14.2.SPD.1WSI40125, LEHR.DRV.MTR..M15.1.SPD.1WSI40126, LEHR.DRV.MTR..M15.2.SPD.1WSI40127, LEHR.DRV.MTR..M16.1.SPD.1WSI40128, DRIVE.

## 컬럼 삭제 작업
1. right 공정에 해당하는 컬럼 삭제
    - 공통공정, Left공정, Right공정으로 분류가능함
    - Left공정, Right공정은 유사한 데이터로 추정됨
    - Left공정, Right공정의 피처 중요도는 비슷할 것
    - Right공정을 제외하여 작업

2. 상관계수 0.9 이상인 컬럼
    - 상관계수 0.9 이상인 컬럼 쌍 중 1개만 남기고 모두 삭제


### 삭제할 컬럼 리스트 만들기

In [42]:
# Right 공정에 해당하는 컬럼
col_right = [   'DB.N2.SCREEN.FLOW.RIGHT.1FI49012_PV',
                 'DB.N2.SCREEN.FLOW.RIGHT.1FI49012_PV.1',
                 'Right.edge',
                 'TMP.TIN..BAY.1.RIGHT.1TI30202.PV',
                 'TMP.TIN..BAY.5.RIGHT.1TI30206.PV',
                 'TMP.TIN..BAY.7.RIGHT.1TI30208.PV',
                 'TMP.TIN..BAY.10.RIGHT.1TI30210.PV',
                 'TMP.GLASS..RIGHT.EXIT.1TIC30111.PV',
                 'X.10.BAY.RIGHT.PRESSURE.1CLBAY10RIGHT_CPV',
                 'X.1.BAY.RIGHT.PRESSURE.1CLBAY1RIGHT_CPV',
                 'X.7.BAY.RIGHT.PRESSURE.1CLBAY7RIGHT_CPV',
                 'RET.AMBIANT.RIGHT.TMP.1TI42603.PV',
                 'BATH.AMBIENT.3.BAY.RIGHT.1TI31002_PV',
                 'EXIT.LIP.PLATE.RIGHT.1TI30605.PV',
                 'E.L.DB.RIGHT.N2.FLOW.1FI33502.PV',
                 'DROSS.BOX.N2.BTM.HT1.R.1JI39006.PV',
                 'DROSS.BOX.N2.BTM.HT..R..1JI39010.PV',
                 'DROSSBOX.N2.SCN.R.1JI39012_PV',
                 'HOOD.N2.HEATER.R..1JI39008.PV',
                 'DDP.RIGHT.N2.H2.9..1FI33421.PV',
                 'SPOUT.RIGHT.UPSTREAM.1FI33516.PV',
                 'SPOUT.RIGHT.BOS.UPSTREAM.1FI33517.PV',
                 'SPOUT.RIGHT.BOTTOM.1FI33518.PV',
                 'SPOUT.RIGHT.TOP.1FI33519.PV',
                 'LOR.1.R.Axis',
                 'LOR.1.R.Hori',
                 'LOR.2.R.Axis',
                 'LOR.2.R.Hori',
                 'LOR.3.R.Axis',
                 'LOR.3.R.Hori',
                 'r_tin_bay_1_5',
                 'r_tin_bay_5_7',
                 'r_tin_bay_7_10',
                 'r_top_n.1_mean',
                 'r_top_n.2_mean',
                 'r_n.2_glass',
                 'D_AB1_R_UL',
                 'D_AB2_R_UL',
                 'D_AB3_R_UL',
                 'D_AB4_R_UL',
                 'D_AB5_R_UL',
                 'D_AB6_R_UL',
                 'D_AB7_R_UL',
                 'D_AB8_R_UL',
                 'D_AB9_R_UL',
                 'D_AB10_R_UL',
                 'D_AB11_R_UL',
                 'D_C1_R_UL',
                 'D_C2_R_UL',
                 'D_C3_R_UL',
                 'D_C4_R_UL',
                 'S_AB1_AB2_U_R',
                 'S_AB1_AB2_L_R',
                 'S_AB2_AB3_U_R',
                 'S_AB2_AB3_L_R',
                 'S_AB3_AB4_U_R',
                 'S_AB3_AB4_L_R',
                 'S_AB4_AB5_U_R',
                 'S_AB4_AB5_L_R',
                 'S_AB5_AB6_U_R',
                 'S_AB5_AB6_L_R',
                 'S_AB6_AB7_U_R',
                 'S_AB6_AB7_L_R',
                 'S_AB7_AB8_U_R',
                 'S_AB7_AB8_L_R',
                 'S_AB8_AB9_U_R',
                 'S_AB8_AB9_L_R',
                 'S_AB9_AB10_U_R',
                 'S_AB9_AB10_L_R',
                 'S_AB10_AB11_U_R',
                 'S_AB10_AB11_L_R',
                 'S_AB11_C1_U_R',
                 'S_AB11_C1_L_R',
                 'S_AB1_AB11_R',
                 'S_AB11_AB7_R',
                 'S_AB7_C4_R',
                 'S_C1_C2_U_R',
                 'S_C1_C2_L_R',
                 'S_C2_C3_U_R',
                 'S_C2_C3_L_R',
                 'S_C3_C4_U_R',
                 'S_C3_C4_L_R',
                 'S_C4_RET_R',
                 'S_RET_F_R',
                 'STD_U_R',
                 'STD_L_R',
                 'STD_GLS_R',
                 '투입.RIGHT.',
                 'ROLLDOWN.폐기.RIGHT.',
                 'ROLLDOWN.진공패드.폐기율..RIGHT.',
                 '진공패드.폐기.RIGHT.']
# 개수 확인
len(col_right) # 91개

91

In [43]:
# 모든 컬럼에 대해 상관계수를 구하고 상관계수가 0.9 이상인 컬럼 중 1개만 남기고 모두 제거
import numpy as np

# 모든 컬럼에 대해 상관계수의 절대값 구하기
corr_arr = abs(df_origin.corr())

# 상관계수가 0.9 이상인 컬럼 중 1개만 남기고 모두 제거
col_corr = []
for i, col in enumerate(corr_arr.columns):
    for j in range(len(corr_arr.index)):
        if i == j:
            continue
        elif corr_arr.index[i] in col_corr:           
            break
        # print(i, j, corr_arr.index[j], np.abs(corr_arr.iloc[j,i]))
        if np.abs(corr_arr.iloc[j,i]) >= 0.9:
            col_corr.append(corr_arr.index[j])
            # print(corr_arr.index[j], '-->', corr_arr.index[i])
# 상관계수 0.9이상이라 지워질 컬럼 수 확인
len(col_corr)

685

In [44]:
# 두 리스트 합침
col_corr.extend(col_right)
print(len(col_corr))

# 중복 제거
del_list = list(set(col_corr))
print(len(del_list))

776
504


### 컬럼 삭제 수행

In [45]:
# 전체 컬럼
# 원본에서 결측치 컬럼 삭제하고 남은 목록
col_tot = fact_data.columns.tolist()
len(col_tot)

833

In [46]:
# 컬럼 삭제 수행
col_use = []
for col in col_tot:
    if col not in del_list:
        col_use.append(col)

# 확인
len(col_use)

329

In [47]:
col_use

['LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101',
 'X1WOS45101_PV',
 'X1WOS45105_PV',
 'X1WOS45106_PV',
 'X1WOS45107_PV',
 'X1WOS45109_PV',
 'DCS_Offset_RPM_M11.1WOS45111_PV',
 'DCS_Offset_RPM_M15.1WOS45115_PV',
 'DCS_Offset_RPM_M16.1WOS45116_PV',
 'DAMP.POSITION.0.BAY.CTRL.1TIC33420.PV',
 'DAMP.POSITION.1.BAY.CTRL.1TIC33421.PV',
 'DB.N2.SCREEN.FLOW.LEFT.1FI49011_PV',
 'N2.TO.EA1313A.1FI33503A_PV',
 'ROOF.HEATING.Zone.1.1JIC31101.PV',
 'ROOF.HEATING.Zone.2.1JIC31102.PV',
 'ROOF.HEATING.Zone.3.1JIC31103.PV',
 'ROOF.HEATING.Zone.4.1JIC31104.PV',
 'ROOF.HEATING.Zone.7.1JIC31107.PV',
 'ROOF.HEATING.Zone.8.1JIC31108.PV',
 'ROOF.HEATING.Zone.9.1JIC31109.PV',
 'ATM.FLOW.SECT.8.N2.H2.1FI33414.PV',
 'ATM.FLOW.SECT.9.N2.1FIC33415.PV',
 'cluster',
 'Gross.width',
 'Left.edge',
 'UPPER.AB1.Z1.TMP.1TIC41101.PV',
 'UPPER.AB1.Z6.TMP.1TIC41106.PV',
 'LOWER.AB1.Z9.TMP.1TIC41152.PV',
 'UPPER.AB2.Z20.TMP.1TIC41206.PV',
 'LOWER.AB3.Z37.TMP.1TIC41352.PV',
 'UPPER.AB5.Z57.TMP.1TIC41501.PV',
 'LOWER.AB5.Z67.TMP.1TIC4155

## col_use의 컬럼 이용해서 작업 수행하기